# Определение уровня сложности англоязычных фильмов

Нам поставлена задача разработать ML решение для автоматического определения уровня сложности англоязычных фильмов.

Нам предоставлены следующие данные:
- два pdf_файла, которые содержат списки слов, распределённые по сложности текста на группы A2, B1, B2, C1;
- xlsx-файл, содержащий названия фильмов в одном столбце, и категорию сложности каждого фильма в другом столбце;
- папку `Subtitles_all`, которая содержит пять папок `A2`, `B1`, `B2`, `C1` и `Subtitles`. Каждая папка содержит файлы в формате `.srt`, именем файла является название фильма, а содержание файла - текст субтитров фильма.

Нам предстоит изучить предоставленные данные и разработать ML решение, которое будет предсказывать категорию сложности англоязычных фильмов по субтитрам. 

## Загрузка необходимых библиотек и инструментов. Загрузка предоставленных файлов и папок. Предобработка данных.

### Загрузим необходимые библиотеки и инструменты.

In [1]:
import PyPDF2
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import chardet
from tqdm import tqdm
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
import joblib

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Обработка pdf-файлов - словарей, содержащих английский слова, разбитые на категории сложности.

**Из предоставленных файлов извлечём словари по категориям и сохраним в отдельных переменных.**

In [2]:
def extract_text_with_pypdf2(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def extract_words_between(text, start_word, end_word):
    lines = text.splitlines()
    start_index = next((i for i, line in enumerate(lines) if line.startswith(start_word)), None)
    end_index = next((i for i, line in enumerate(lines) if end_word in line), None)

    if start_index is not None and end_index is not None and end_index > start_index:
        extracted_lines = lines[start_index:end_index]
        words = [line.split()[0] for line in extracted_lines]
        if end_word in lines[end_index]:
            words.append(end_word)
        return words
    return []

if __name__ == "__main__":
    pdf_file_path_1 = 'D:/Яндекс Практикум/17. Мастерская_2/Датасет/The_Oxford_3000_by_CEFR_level.pdf'
    pdf_file_path_2 = 'D:/Яндекс Практикум/17. Мастерская_2/Датасет/The_Oxford_5000_by_CEFR_level.pdf'

    # Extract text using PyPDF2 for the first PDF
    text_pypdf2_1 = extract_text_with_pypdf2(pdf_file_path_1)

    A2_words = extract_words_between(text_pypdf2_1, "ability", "zero")

    print("Словарь A2_words:")
    print("Первые слова словаря: ", A2_words[:5])
    print("Последние слова словаря: ", A2_words[-5:])
    print()

    B1_words = extract_words_between(text_pypdf2_1, "absolutely", "youth")

    print("Словарь B1_words:")
    print("Первые слова словаря: ", B1_words[:5])
    print("Последние слова словаря: ", B1_words[-5:])
    print()

    B2_words_1 = extract_words_between(text_pypdf2_1, "abandon", "zone")

    text_pypdf2_2 = extract_text_with_pypdf2(pdf_file_path_2)

    B2_words_2 = extract_words_between(text_pypdf2_2, "absorb", "wrist")

    B2_words = B2_words_1 + B2_words_2

    print("Словарь B2_words:")
    print("Первые слова словаря: ", B2_words[:5])
    print("Последние слова словаря: ", B2_words[-5:])
    print()

    C1_words = extract_words_between(text_pypdf2_2, "abolish", "yield")

    print("Словарь C1_words:")
    print("Первые слова словаря: ", C1_words[:5])
    print("Последние слова словаря: ", C1_words[-5:])
    print()

Словарь A2_words:
Первые слова словаря:  ['ability', 'able', 'abroad', 'accept', 'accident']
Последние слова словаря:  ['worst', 'wow', 'yet', 'yours', 'zero']

Словарь B1_words:
Первые слова словаря:  ['absolutely', 'access', 'accommodation', 'account', 'achievement']
Последние слова словаря:  ['written', 'wrong', 'yard', 'young', 'youth']

Словарь B2_words:
Первые слова словаря:  ['abandon', 'absolute', 'academic', 'acceptable', 'accompany']
Последние слова словаря:  ['workforce', 'workplace', 'workshop', 'worm', 'wrist']

Словарь C1_words:
Первые слова словаря:  ['abolish', 'abortion', 'absence', 'absent', 'absurd']
Последние слова словаря:  ['worship', 'worthwhile', 'worthy', 'yell', 'yield']



### Создаём датафрейм, содержащий два столбца - название файла и текст субтитров из этого файла.

**Импортируем файлы субтитров из папок, в которых они содержатся. Определяем кодировку файла, открываем файл и извлекаем текст субтитров, создаём датафрейм из названий файлов и субтитров, содержащихся во всех папках с субтитрами, попутно удаляем пустые строки, временные коды и номер субтитра.**

In [3]:
# Функция для определения кодировки файла
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Инициализируем пустой список для хранения данных о субтитрах
subtitles_data = []
errors = []

# Путь к базовой папке с файлами SRT
base_folder = 'D:/Яндекс Практикум/17. Мастерская_2/Датасет/Subtitles_all'

# Цикл для обхода папок с файлами SRT
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)
    if os.path.isdir(folder_path):  # Проверяем, что это директория
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if file_path.endswith(".srt"):  # Проверяем, что это файл SRT
                try:
                    # Определяем кодировку файла
                    encoding = detect_encoding(file_path)

                    # Открываем файл SRT с определенной кодировкой
                    with open(file_path, 'r', encoding=encoding) as file:
                        subtitle_lines = file.readlines()

                    # Обрабатываем файл SRT и извлекаем текст субтитров
                    subtitles = []
                    current_subtitle = ""
                    for line in subtitle_lines:
                        line = line.strip()
                        if line.isdigit() or line == "":
                            # Пропускаем номер субтитра и пустые строки
                            continue
                        elif "-->" in line:
                            # Пропускаем временные коды
                            continue
                        else:
                            current_subtitle += " " + line

                            if line.endswith("."):
                                # Конец субтитра, добавляем его в список
                                subtitles.append(current_subtitle.strip())
                                current_subtitle = ""

                    # Добавляем данные из файла SRT в список
                    if subtitles:
                        subtitles_data.append({'file': filename[:-4], 'text': ' '.join(subtitles)})
                except Exception as e:
                    errors.append((file_path, e))

# Создаем DataFrame из списка данных о субтитрах
df = pd.DataFrame(subtitles_data)

# Выводим первые строки DataFrame для проверки
display(df.head())

print("Ошибки в следующих файлах:")
for file_path, error in errors:
    print(f"{file_path}: {error}")

,file,text
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ..."


Ошибки в следующих файлах:


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    276 non-null    object
 1   text    276 non-null    object
dtypes: object(2)
memory usage: 4.4+ KB


**На случайном файле смотрим, что субтитры импортировались нормально.**

In [5]:
print(df['text'].sample())

183    Created and Encoded by --  Bokutox -- of  www....
Name: text, dtype: object


### Импортируем файл xlsx, создадим датафрейм с названием файла и уровнем сложности текста.

In [6]:
# Путь к файлу .xlsx
file_path = 'D:/Яндекс Практикум/17. Мастерская_2/Датасет/movies_labels.xlsx'

# Импорт таблицы Excel и создание DataFrame
df_xlxs = pd.read_excel(file_path)

# Вывод первых 5 строк DataFrame для проверки
df_xlxs = df_xlxs.drop(['id'], axis=1)
df_xlxs.columns = df_xlxs.columns.str.lower()
df_xlxs.head()

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


**Некоторые файлы имеют не один, а два-три уровня сложности. Упростим таблицу - оставим только наименьший уровень сложности.**

In [7]:
df_xlxs['level'] = df_xlxs['level'].str.slice(0, 2)
df_xlxs.head()

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2


## Предобработка данных.

### Обработаем текст субтитров.

**Импортируем словарь, содержащий стоп-слова.**

In [8]:
stop_words = stopwords.words('english')
lmtzr = WordNetLemmatizer()

**Удаляем лишние символы, стоп-слова. Методом лемматизации приводим слова к словарной форме.**

In [9]:
# Функции предобработки и лемматизации текста
del_n = re.compile('\n')
del_tags = re.compile('<[^>]*>')
del_brackets = re.compile('\([^)]*\)')
clean_text = re.compile('[^а-яa-z\s]')
del_spaces = re.compile('\s{2,}')

def prepare_text(text):
    text = del_n.sub(' ', str(text).lower())
    text = del_tags.sub(' ', text)
    text = del_brackets.sub('', text)
    res_text = clean_text.sub('', text)
    return del_spaces.sub(' ', res_text)

def del_stopwords(text):
    stop_words = set(stopwords.words('english'))
    clean_tokens = tuple(map(lambda x: x if x not in stop_words else '', word_tokenize(text)))
    res_text = ' '.join(clean_tokens)
    return res_text

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

def lemmatize(text):
    lmtzr = WordNetLemmatizer()
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    lemmatized_words = [lmtzr.lemmatize(word, pos=get_wordnet_pos(tag)) for word, tag in tagged_words]
    return ' '.join(lemmatized_words)

In [10]:
# Применяем предобработку и лемматизацию к тексту
df['processed_text'] = df['text'].apply(prepare_text)
df['clean_text'] = df['processed_text'].apply(del_stopwords)
df['lemmatized_text'] = df['clean_text'].apply(lemmatize)

display(df.head())

,file,text,processed_text,clean_text,lemmatized_text
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...


**Переименуем столбец с названием файла**

In [11]:
df.columns = df.columns.str.replace('file', 'movie')

In [12]:
df.head()

,movie,text,processed_text,clean_text,lemmatized_text
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...


**Объединяем таблицы с текстом и категорией сложности по названию файла, создаём новую таблицу**

In [13]:
df_1 = pd.merge(df, df_xlxs, how ='outer', on ='movie')

In [14]:
df_1.head()

,movie,text,processed_text,clean_text,lemmatized_text,level
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2


**Проверяем таблицу на пропуски в столбце `text` и удаляем пустые строки.**

In [15]:
df_1.isnull().sum()

movie               0
text               11
processed_text     11
clean_text         11
lemmatized_text    11
level              50
dtype: int64

**Удаляем пустые строки в столбце `text`.**

In [16]:
df_1 = df_1.dropna(subset=['text'])

In [17]:
df_1.head()

,movie,text,processed_text,clean_text,lemmatized_text,level
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2


In [18]:
df_1['text'].isnull().sum()

0

**Исследуем столбец `level`.**

In [19]:
# Выводим строки с пропущенными значениями в столбце 'level'
rows_with_missing_values = df_1[df_1['level'].isnull()]

rows_with_missing_values.head()

,movie,text,processed_text,clean_text,lemmatized_text,level
19,SlingShot (2014) WEB.eng,Here's something that should hurt your brain. ...,heres something that should hurt your brain we...,heres something hurt brain empty half ...,here something hurt brain empty half bed hospi...,NaN
22,Valentine`s.Day.2010.Subtitles.YIFY,"<i>Hey there, all you sleepy Angelenos. A good...",hey there all you sleepy angelenos a good goo...,hey sleepy angelenos good good morning ...,hey sleepy angelenos good good morning buddy r...,NaN
23,Angela`s.Christmas.2018.WEBRip.Netflix,"[match snaps, fizzles] [fire crackling] [puffs...",match snaps fizzles fire crackling puffs wagon...,match snaps fizzles fire crackling puffs wagon...,match snap fizzle fire crackle puff wagon whee...,NaN
41,"Crown, The S01E01 - Wolferton Splash.en.SDH",[coughing] [coughing continues] [spits] [exhal...,coughing coughing continues spits exhales deep...,coughing coughing continues spits exhales deep...,cough cough continue spit exhales deeply toile...,NaN
42,"Crown, The S01E01 - Wolferton Splash.en","In seeking his British nationalization, His Ro...",in seeking his british nationalization his roy...,seeking british nationalization royal highn...,seek british nationalization royal highness pr...,NaN


**Заполняем пропуски в столбце `level`, для этого пишем функцию, которая заполняет пропущенное значение названием папки, которое и является значением категории сложности. Исключение составляют файлы, находящиеся в папке `Subtitles`, эти строки удаляем.**

In [20]:
# Задаем путь к базовой папке с файлами SRT
base_folder = 'D:/Яндекс Практикум/17. Мастерская_2/Датасет/Subtitles_all'

# Функция для получения названия папки из полного пути к файлу
def get_folder_name(file_path):
    folder_path = os.path.dirname(file_path)
    return os.path.basename(folder_path)

# Создаем словарь с соответствием названий файлов без расширения и папок
filename_to_folder = {}
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)
    if os.path.isdir(folder_path):  # Проверяем, что это директория
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if file_path.endswith(".srt"):  # Проверяем, что это файл SRT
                filename_without_extension = os.path.splitext(filename)[0]
                filename_to_folder[filename_without_extension] = folder_name

# Заполняем пропуски в столбце 'level' соответствующим значением из словаря
df_1['level'] = df_1['level'].fillna(df_1['movie'].apply(lambda x: filename_to_folder.get(x, '')))

# Выводим первые несколько строк для проверки
df_1.head(20)

,movie,text,processed_text,clean_text,lemmatized_text,level
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2
5,The Walking Dead-S01E06-TS-19.English,- ( people yelling ) - ( radio chatter ) Hey h...,hey hey whoa whoa whoa whoa maam maam please ...,hey hey whoa whoa whoa whoa maam maam please p...,hey hey whoa whoa whoa whoa maam maam please p...,A2
6,AmericanBeauty1999.BRRip,"I need a father who's a role model, not some h...",i need a father whos a role model not some hor...,need father whos role model horny geek bo...,need father whos role model horny geek boy who...,B1
7,Angelas.Christmas.Wish.2020,"Where are we going? Come on, Angela. But who'l...",where are we going come on angela but wholl mi...,going come angela wholl mind sheep dada ...,go come angela wholl mind sheep dada want hear...,B1
8,Indiana Jones And The Last Crusade DVDRip Xvid...,"Dismount! Herman's horse-sick! Chaps, no one w...",dismount hermans horsesick chaps no one wander...,dismount hermans horsesick chaps one wander ...,dismount herman horsesick chap one wander pass...,B1
9,mechanic-resurrection_,"Mr. Santos, so good to see you. We saved your ...",mr santos so good to see you we saved your usu...,mr santos good see saved usual table mr s...,mr santos good see save usual table mr santos ...,B1


In [21]:
df_1.isnull().sum()

movie              0
text               0
processed_text     0
clean_text         0
lemmatized_text    0
level              0
dtype: int64

In [22]:
# Подсчитываем уникальные значения в столбце 'level' и их количество
level_counts = df_1['level'].value_counts()

# Выводим количества строк со значением 'subtitles' в столбце 'level'
print(level_counts['Subtitles'])

7


In [23]:
df_sub = df_1[df_1['level'] == 'Subtitles']
df_sub

,movie,text,processed_text,clean_text,lemmatized_text,level
178,BrenВ.Brown.The.Call.to.Courage.2019.720.NF.72...,[presenter] <i>She spent 20 years studying cou...,presenter she spent years studying courage vul...,presenter spent years studying courage vulner...,presenter spend year study courage vulnerabili...,Subtitles
182,Casper,[Music] [Music] okay one picture in my history...,music music okay one picture in my history thi...,music music okay one picture history im af...,music music okay one picture history im afraid...,Subtitles
200,Gogo_Loves_English,[Music] so [Music] wow [Music] hello hello my ...,music so music wow music hello hello my names ...,music music wow music hello hello names tony...,music music wow music hello hello name tony wh...,Subtitles
203,Harry_Potter_and_the_philosophers_stone(2001),"I should've known that you would be here, Prof...",i shouldve known that you would be here profes...,shouldve known would professor mcgonagall...,shouldve know would professor mcgonagall good ...,Subtitles
235,Pride_and_Prejudice,[Music] so [Music] um [Music] [Music] so [Musi...,music so music um music music so music lydia k...,music music um music music music lydia kitty...,music music um music music music lydia kitty m...,Subtitles
249,The_Ghost_Writer,[Music] [Music] so [Music] [Music] [Music] [Mu...,music music so music music music music applaus...,music music music music music music applause ...,music music music music music music applause h...,Subtitles
276,Westworld_scenes_of_Dr_Robert_Ford,[Music] no one's complained [Music] there's th...,music no ones complained music theres the lady...,music ones complained music theres lady wh...,music one complain music there lady white shoe...,Subtitles


In [24]:
df_1 = df_1[df_1.level != 'Subtitles']

In [25]:
df_1 = df_1.reset_index(drop=True)

In [26]:
df_1.head()

,movie,text,processed_text,clean_text,lemmatized_text,level
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2


**Проверяем, что в таблице остались только заполненные строки, столбец `level` содержит корректные значения.**

In [27]:
df_1['level'].unique()

array(['A2', 'B1', 'B2', 'C1'], dtype=object)

### Создаём новые признаки.

**Для создания новых признаков считаем вхождение слов каждой категории в текст субтитров, количество слов в тексте, а также процент слов кждой категории в тексте.**

In [28]:
# Создаем функцию для подсчета количества вхождений слов в текст
def count_words(text, word_list):
    word_count = sum(1 for word in text.split() if word in word_list)
    return word_count

# Применяем функцию для каждого текста и каждого списка слов
df_1['A2_word_count'] = df_1['lemmatized_text'].apply(lambda x: count_words(x, A2_words))
df_1['B1_word_count'] = df_1['lemmatized_text'].apply(lambda x: count_words(x, B1_words))
df_1['B2_word_count'] = df_1['lemmatized_text'].apply(lambda x: count_words(x, B2_words))
df_1['C1_word_count'] = df_1['lemmatized_text'].apply(lambda x: count_words(x, C1_words))

# Добавляем столбец с общим количеством слов в тексте
#df_1['total_word_count'] = df_1['lemmatized_text'].apply(lambda x: len(x))
df_1['total_word_count'] = df_1['lemmatized_text'].apply(lambda x: len(x.split()))

# Добавляем столбцы с долей слов каждой категории в тексте
df_1['A2_word_%'] = df_1['A2_word_count'] / df_1['total_word_count']
df_1['B1_word_%'] = df_1['B1_word_count'] / df_1['total_word_count']
df_1['B2_word_%'] = df_1['B2_word_count'] / df_1['total_word_count']
df_1['C1_word_%'] = df_1['C1_word_count'] / df_1['total_word_count']

df_1.head()

,movie,text,processed_text,clean_text,lemmatized_text,level,A2_word_count,B1_word_count,B2_word_count,C1_word_count,total_word_count,A2_word_%,B1_word_%,B2_word_%,C1_word_%
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2,261,152,142,65,1676,0.155728,0.090692,0.084726,0.038783
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2,246,163,156,39,1686,0.145907,0.096679,0.092527,0.023132
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2,368,151,181,74,2170,0.169585,0.069585,0.083410,0.034101
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2,328,188,169,46,1932,0.169772,0.097308,0.087474,0.023810
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2,268,170,129,57,1597,0.167815,0.106450,0.080776,0.035692


**Масштабируем все числовые признаки.**

In [29]:
numeric = ['A2_word_count', 'B1_word_count', 'B2_word_count',
           'C1_word_count', 'total_word_count', 'A2_word_%', 'B1_word_%', 'B2_word_%', 'C1_word_%']

In [30]:
scaler = StandardScaler()
scaler.fit(df_1[numeric])

StandardScaler()

In [31]:
pd.options.mode.chained_assignment = None
df_1[numeric] = scaler.transform(df_1[numeric])
df_1.head()

,movie,text,processed_text,clean_text,lemmatized_text,level,A2_word_count,B1_word_count,B2_word_count,C1_word_count,total_word_count,A2_word_%,B1_word_%,B2_word_%,C1_word_%
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering ) ( brakes squeak ) - ( engi...,little girl im a policeman little girl dont b...,little girl im policeman little girl dont af...,little girl im policeman little girl dont afra...,A2,-1.351579,-1.393482,-1.389824,-1.241052,-1.289059,-0.427852,-0.483181,-0.175686,0.366513
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping ) - ( bugs chittering ) Boy...,boy mom right here any luck how do we tell if...,boy mom right luck tell theyre poison uh...,boy mom right luck tell theyre poison uh there...,A2,-1.410225,-1.320338,-1.279023,-1.721184,-1.282792,-0.886396,-0.084365,0.295649,-1.584313
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling ) Merle: That's right. You ...,merle thats right you heard me bitch you got ...,merle thats right heard bitch got problem ...,merle thats right hear bitch get problem bring...,A2,-0.933233,-1.400132,-1.081164,-1.074852,-0.979498,0.219185,-1.889329,-0.255160,-0.217001
3,The Walking Dead-S01E04-Vatos.English,( birds chirping ) - What? - Nothing. It's not...,what nothing its not nothing its always somet...,nothing nothing always something didnt dad...,nothing nothing always something didnt dad tea...,A2,-1.089624,-1.154103,-1.176136,-1.591918,-1.128639,0.227916,-0.042397,-0.009620,-1.499822
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks ) - Rick: Morgan, I ...",rick morgan i dont know if youre out there i ...,rick morgan dont know youre dont know ...,rick morgan dont know youre dont know hear may...,A2,-1.324210,-1.273792,-1.492710,-1.388785,-1.338563,0.136510,0.566587,-0.414284,-0.018749


In [32]:
df_1.corr()

,A2_word_count,B1_word_count,B2_word_count,C1_word_count,total_word_count,A2_word_%,B1_word_%,B2_word_%,C1_word_%
A2_word_count,1.000000,0.945672,0.923453,0.867351,0.966391,-0.072345,-0.069939,-0.249865,-0.096646
B1_word_count,0.945672,1.000000,0.931060,0.849370,0.934342,-0.144309,0.151579,-0.183528,-0.058388
B2_word_count,0.923453,0.931060,1.000000,0.836127,0.893735,-0.096822,0.063359,0.023103,-0.006543
C1_word_count,0.867351,0.849370,0.836127,1.000000,0.887569,-0.217707,-0.073763,-0.232509,0.285164
total_word_count,0.966391,0.934342,0.893735,0.887569,1.000000,-0.259290,-0.170055,-0.361024,-0.124797
A2_word_%,-0.072345,-0.144309,-0.096822,-0.217707,-0.259290,1.000000,0.407702,0.549889,0.082971
B1_word_%,-0.069939,0.151579,0.063359,-0.073763,-0.170055,0.407702,1.000000,0.531400,0.319421
B2_word_%,-0.249865,-0.183528,0.023103,-0.232509,-0.361024,0.549889,0.531400,1.000000,0.266357
C1_word_%,-0.096646,-0.058388,-0.006543,0.285164,-0.124797,0.082971,0.319421,0.266357,1.000000


**Выбираем наименее коррелирующие между собой признаки, которые будут участвовать в обучении модели - это признаки процентного соотношения.**

### Выбор модели обучения.

**Обучающей моделью выбираем  Многоклассовую Логистическую регрессию.**

**Делим данные на обучающие и целевой признак, внутри кросс-валидации делим данные на тренировочную, валидационную и тестовую выборки, подбираем гиперпараметры, производим обучение, тестирование модели и выбираем модель с лучшими гиперпараметрами. Метрикой качества выбираем Accuracy.**

In [33]:
X_text = df_1['lemmatized_text']
X_num = df_1[['A2_word_%', 'B1_word_%', 'B2_word_%', 'C1_word_%']]
y = df_1['level']

# Создаем экземпляр StratifiedKFold для кросс-валидации
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Инициализируем пустой список для хранения результатов
results = []
best_test_params = 0
best_params_gs_lr = []

# Итерируемся по разбиениям кросс-валидации
for trainval_index, test_index in tqdm(cv.split(X_text, y), total=cv.get_n_splits()):
    X_text_trainval, X_text_test = X_text.iloc[trainval_index], X_text.iloc[test_index]
    X_num_trainval, X_num_test = X_num.iloc[trainval_index], X_num.iloc[test_index]
    y_trainval, y_test = y.iloc[trainval_index], y.iloc[test_index]

    # Дополнительно разбиваем временную выборку на тренировочную и валидационную
    X_text_train, X_text_val, X_num_train, X_num_val, y_train, y_val = train_test_split(
        X_text_trainval, X_num_trainval, y_trainval, test_size=0.25, random_state=42)

    # Создаем TfidfVectorizer для текстовых данных
    tfidf_vect = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True,
                                 strip_accents=None, lowercase=False, preprocessor=None)
    X_text_train_tfidf = tfidf_vect.fit_transform(X_text_train)
    X_text_val_tfidf = tfidf_vect.transform(X_text_val)
    X_text_test_tfidf = tfidf_vect.transform(X_text_test)

    # Объединяем текстовые и числовые признаки для текущего фолда
    X_train_combined = hstack([X_text_train_tfidf, X_num_train])
    X_val_combined = hstack([X_text_val_tfidf, X_num_val])
    X_test_combined = hstack([X_text_test_tfidf, X_num_test])

    # Создаем мультиклассовую логистическую регрессию
    lr = LogisticRegression(random_state=42, multi_class='ovr')

    # Задаем сетку параметров для поиска по сетке
    param_grid = [{'penalty': ['l1', 'l2'],
                   'C': [1.0, 10.0, 100.0],
                   'solver': ['lbfgs', 'liblinear']}]

    # Инициализируем GridSearchCV
    gs_lr_tfidf = GridSearchCV(lr, param_grid,
                               scoring='accuracy',
                               cv=5,
                               verbose=1,
                               n_jobs=-1)

    # Обучаем модель на тренировочных данных с использованием GridSearchCV
    gs_lr_tfidf.fit(X_train_combined, y_train)

    # Получаем лучшие параметры и оцениваем производительность модели на валидационной выборке
    best_params = gs_lr_tfidf.best_params_
    print("Best Parameters:", best_params)

    # Делаем предсказания на валидационных данных
    y_pred = gs_lr_tfidf.predict(X_val_combined)

    # Оцениваем производительность модели на валидационной выборке
    accuracy = accuracy_score(y_val, y_pred)
    print("Accuracy:", accuracy)

    # Выводим отчет по каждому целевому признаку
    print(classification_report(y_val, y_pred))

    # Делаем предсказания на тестовой выборке и оцениваем качество модели
    y_pred_test = gs_lr_tfidf.predict(X_test_combined)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy:", accuracy_test)
    print(classification_report(y_test, y_pred_test))

    # Добавляем результаты в список
    results.append(accuracy)

    if accuracy_test > best_test_params:
        best_test_params = accuracy_test
        best_params_gs_lr = gs_lr_tfidf.best_params_

# Анализируем результаты кросс-валидации
average_accuracy = np.mean(results)
print("Average Accuracy:", average_accuracy)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


 20%|████████████████▊                                                                   | 1/5 [00:13<00:55, 13.94s/it]

Best Parameters: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.6909090909090909
              precision    recall  f1-score   support

          A2       0.50      0.25      0.33         4
          B1       0.56      0.60      0.58        15
          B2       0.75      0.75      0.75        28
          C1       0.78      0.88      0.82         8

    accuracy                           0.69        55
   macro avg       0.65      0.62      0.62        55
weighted avg       0.68      0.69      0.68        55

Test Accuracy: 0.6181818181818182
              precision    recall  f1-score   support

          A2       0.33      0.12      0.18         8
          B1       0.40      0.50      0.44        12
          B2       0.68      0.78      0.72        27
          C1       1.00      0.75      0.86         8

    accuracy                           0.62        55
   macro avg       0.60      0.54      0.55        55
weighted avg       0.61      0.62      0.60        5

 40%|█████████████████████████████████▌                                                  | 2/5 [00:22<00:32, 10.88s/it]

...

Best Parameters: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.5818181818181818
              precision    recall  f1-score   support

          A2       0.00      0.00      0.00         8
          B1       0.50      0.62      0.55        13
          B2       0.59      0.79      0.68        24
          C1       1.00      0.50      0.67        10

    accuracy                           0.58        55
   macro avg       0.52      0.48      0.47        55
weighted avg       0.56      0.58      0.55        55

Test Accuracy: 0.6181818181818182
              precision    recall  f1-score   support

          A2       1.00      0.14      0.25         7
          B1       0.40      0.46      0.43        13
          B2       0.66      0.85      0.74        27
          C1       1.00      0.50      0.67         8

    accuracy                           0.62        55
   macro avg       0.76      0.49      0.52        55
weighted avg       0.69      0.62      0.59        5

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:39<00:09,  9.18s/it]

Best Parameters: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.5636363636363636
              precision    recall  f1-score   support

          A2       0.00      0.00      0.00         7
          B1       0.36      0.38      0.37        13
          B2       0.62      0.77      0.69        26
          C1       0.86      0.67      0.75         9

    accuracy                           0.56        55
   macro avg       0.46      0.46      0.45        55
weighted avg       0.52      0.56      0.54        55

Test Accuracy: 0.7407407407407407
              precision    recall  f1-score   support

          A2       0.75      0.43      0.55         7
          B1       0.67      0.46      0.55        13
          B2       0.72      0.96      0.83        27
          C1       1.00      0.71      0.83         7

    accuracy                           0.74        54
   macro avg       0.78      0.64      0.69        54
weighted avg       0.75      0.74      0.72        5

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:48<00:00,  9.79s/it]

Best Parameters: {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 0.6727272727272727
              precision    recall  f1-score   support

          A2       0.00      0.00      0.00         6
          B1       0.46      0.55      0.50        11
          B2       0.69      0.89      0.78        28
          C1       1.00      0.60      0.75        10

    accuracy                           0.67        55
   macro avg       0.54      0.51      0.51        55
weighted avg       0.63      0.67      0.63        55

Test Accuracy: 0.6481481481481481
              precision    recall  f1-score   support

          A2       0.50      0.14      0.22         7
          B1       0.53      0.67      0.59        12
          B2       0.66      0.78      0.71        27
          C1       1.00      0.62      0.77         8

    accuracy                           0.65        54
   macro avg       0.67      0.55      0.57        54
weighted avg       0.66      0.65      0.63        54

A

In [34]:
print('Best parameter set: %s ' % best_params_gs_lr)
print('Best Accuracy: %.3f' % best_test_params)

Best parameter set: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'} 
Best Accuracy: 0.741


**Сохраняем лучшую модель и TfidfVectorizer для получения предсказаний на новых данных.**

In [35]:
# Получение экземпляра лучшей модели
best_lr_model = gs_lr_tfidf.best_estimator_

# Сохранение модели в файл
model_filename = 'best_logistic_regression_model.joblib'
joblib.dump(best_lr_model, model_filename)

['best_logistic_regression_model.joblib']

In [36]:
# Сохраняем TfidfVectorizer и обученную модель
joblib.dump(tfidf_vect, 'tfidf_vect.pkl')
joblib.dump(gs_lr_tfidf, 'best_lr_model.pkl')

['best_lr_model.pkl']

## Получение предсказаний на новых данных.

**Для предсказания категории сложности фильма загружаем выбранный файл субтитров с разрешением `.srt`. В нашем случае выбираем один из файлов из папки `Subtitles`, которым не была присвоена категория сложности, и которые мы удалили из таблицы до обучения модели.**

In [37]:
tfidf_vect = joblib.load('tfidf_vect.pkl')
loaded_best_lr_model = joblib.load('best_lr_model.pkl')

# Функция для предобработки текста субтитров
def preprocess_subtitle(subtitle_text):
    # Применяем предобработку и лемматизацию как в обучении модели
    processed_text = prepare_text(subtitle_text)
    clean_text = del_stopwords(processed_text)
    lemmatized_text = ''.join(lemmatize(clean_text))
    return lemmatized_text

# Загружаем модель из файла
loaded_best_lr_model = joblib.load(model_filename)

# Путь к файлу .srt
file_path = "D:/Яндекс Практикум/17. Мастерская_2/Датасет/Subtitles_all/Subtitles/The_man_called_Flintstone(1966).srt"

# Определяем кодировку файла
encoding = detect_encoding(file_path)

# Открываем файл .srt с определенной кодировкой
with open(file_path, 'r', encoding=encoding) as file:
    subtitle_lines = file.readlines()

# Обрабатываем файл .srt и извлекаем текст субтитров
subtitles = []
current_subtitle = ""
for line in subtitle_lines:
    line = line.strip()
    if line.isdigit() or line == "":
        # Пропускаем номер субтитра и пустые строки
        continue
    elif "-->" in line:
        # Пропускаем временные коды
        continue
    else:
        current_subtitle += " " + line

        if line.endswith("."):
            # Конец субтитра, добавляем его в список
            subtitles.append(current_subtitle.strip())
            current_subtitle = ""

# Объединяем текст субтитров в одну строку
full_subtitle_text = ' '.join(subtitles)

# Предобрабатываем текст субтитров
preprocessed_subtitle_text = preprocess_subtitle(full_subtitle_text)

# Преобразуем предобработанный текст с помощью ранее обученного TfidfVectorizer
new_text_tfidf = tfidf_vect.transform([preprocessed_subtitle_text])

# Создание массива числовых признаков для новых данных (аналогично тому, что использовалось для обучения модели)
new_A2_word_count = count_words(preprocessed_subtitle_text, A2_words)
new_B1_word_count = count_words(preprocessed_subtitle_text, B1_words)
new_B2_word_count = count_words(preprocessed_subtitle_text, B2_words)
new_C1_word_count = count_words(preprocessed_subtitle_text, C1_words)
new_total_word_count = len(preprocessed_subtitle_text.split())

new_A2_word_percent = new_A2_word_count / new_total_word_count
new_B1_word_percent = new_B1_word_count / new_total_word_count
new_B2_word_percent = new_B2_word_count / new_total_word_count
new_C1_word_percent = new_C1_word_count / new_total_word_count

new_numerical_features = np.array([[new_A2_word_count, new_B1_word_count, new_B2_word_count, new_C1_word_count,
                                  new_total_word_count, new_A2_word_percent, new_B1_word_percent, new_B2_word_percent,
                                  new_C1_word_percent]])

new_numerical_features_selected = new_numerical_features[:, 5:9]

# Объединяем текстовые и числовые признаки
new_combined_features = hstack([new_text_tfidf, new_numerical_features_selected])

# Применяем модель к новым данным
predicted_labels = loaded_best_lr_model.predict(new_combined_features)
print('Фильм соответствует категории сложности английского языка:', predicted_labels)

Фильм соответствует категории сложности английского языка: ['A2']


**Файлу субтитров присвоена категория по результатам предсказания.**

## Вывод.

**Для выполнения поставленной задачи - разработать ML решение для автоматического определения уровня сложности англоязычных фильмов - произвели изучение, предобработку предоставленных данных, создали новые признаки, обучили модель Логистическая регрессия, получили Accuracy 0,741. С помощью этой модели разработали форму и получили предсказание категории сложности англоязычного фильма по тексту субтитров. Задача выполнена.**